In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from tqdm import tqdm
import pickle
import seaborn as sns
import multiprocessing
import re
from importlib import reload

In [2]:
import sys
sys.path.insert(1, '/project/cper_neon_aop/cper_hls_veg_models/biomass/params')
sys.path.insert(1, '/project/cper_neon_aop/hls_nrt/fit')
sys.path.insert(1, '/project/cper_neon_aop/hls_nrt/extract')

In [3]:
#reload(sys.modules["cper_bm_params_fit_year_plot_nobai"])
from cper_bm_params_fit_year_transect import *

In [4]:
#reload(sys.modules["hpc_setup"])
from hpc_setup import launch_dask
from ml_setup import fit_ols, fit_dnn

/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#client.cluster.close()
#client.close()

In [6]:
if backend == 'dask':
    from hpc_setup import launch_dask
    import psutil
    interface_prefs = [
        'ibp175s0',
        'ibp59s0',
        'enp24s0f0',
        'ens7f0']
    interface = [x for x in interface_prefs if x in list(psutil.net_if_addrs().keys())][0]
    if len(interface) == 0:
        print('ERROR: Preferred interfaces not found on node!')
    else:
        print(interface)
    num_jobs=32
    client = launch_dask(cluster_loc=cluster_loc,
                         num_jobs=num_jobs,
                         mem_gb_per=4.0,
                         partition='scavenger', 
                         duration='10:00:00',
                         slurm_opts={'interface': interface},
                        wait_timeout=300,
                        debug=False)
    display(client)
    nthreads = len(client.nthreads())
else:
    nthreads = multiprocessing.cpu_count()

ibp175s0
   setting up cluster on HPC...


<Client: 'tcp://10.1.10.111:42173' processes=0 threads=0, memory=0 B>

In [7]:
import torch
if torch.has_cuda and use_cuda:
    device='cuda'
    print(torch.cuda.device_count())
else:
    device='cpu'
print(device)

cpu


In [8]:
reload(sys.modules["ml_setup"])
from ml_setup import make_model_dictionary
from ml_setup import run_ml_models_bootstrap_yr

In [9]:
with open('./results/tmp/ml_train_cper_bm_transect_cv_Year_tuneby_Year_results.pk', 'rb') as f:
    mod_dict = pd.compat.pickle_compat.load(f)

In [10]:
df = load_df(inPATH, ['Date', 'Date_mean'])

In [11]:
for k in mod_dict:
    if k not in ['CPER_2022', 'OLS_2022', 'OLS', 'DNN']:
        param_best = {}
        param_best_dict = {p: [] for p in mod_dict[k]['param_grid'].keys()}
        for y in mod_dict[k]['tune_results'].keys():
            param_best_dict_tmp = mod_dict[k]['tune_results'][y]['params'][mod_dict[k]['tune_results'][y]['mean_test_' + mod_dict[k]['tune_refit']].argmax()]
            for p in param_best_dict_tmp.keys():
                param_best_dict[p].append(param_best_dict_tmp[p])
        #print(param_best_dict)
        for p in param_best_dict:
            if all([type(i) in [float, np.float64] for i in param_best_dict[p]]):
                param_best[p] = np.mean(param_best_dict[p])
            elif all([type(i) in [int, np.int64] for i in param_best_dict[p]]):
                param_best[p] = int(np.mean(param_best_dict[p]))
            elif all([i is None for i in param_best_dict[p]]):
                param_best[p] = None
            else:
                print('ERROR')
        mod_dict[k]['param_best'] = param_best

/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [12]:
mod_dict['DNN']['fit'] = False
mod_dict['CPER_2022']['fit'] = False

In [13]:
if 'BAI_236' not in var_names or 'NDII7' not in var_names or 'NIR1' not in var_names:
    mod_dict['OLS_2022']['fit'] = False

In [14]:
if 'transect' in inPATH:
    df['Plot'] = df[id_col].transform(lambda x: '_'.join(x.split('_')[:-1]))
    agg_plot=True
else:
    agg_plot=False

In [15]:
#client.restart()
#import distributed
#client = [x for x in distributed.client._global_clients.values()][0]
#display(client)
#nthreads = len(client.nthreads())

In [16]:
#reload(sys.modules["ml_setup"])
#from ml_setup import run_ml_models_bootstrap_yr

In [ ]:
df_out = run_ml_models_bootstrap_yr(nickname, mod_dict, df, y_col, date_col, var_names,
                               tuneby_group,
                               backend, nthreads,
                               cper_mod_xfrm, cper_mod_xfrm_func, client,
                               retune_bootstrap=True,
                                    agg_plot=agg_plot,
                                   save_path='./results/tmp/vor_bootstrap_by_year_transect.csv')

Running 3-year combos


100%|██████████| 120/120 [00:42<00:00,  2.80it/s]


Running 4-year combos


  0%|          | 0/210 [00:00<?, ?it/s]/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.194e+01, tolerance: 1.723e+01
  model = cd_fast.enet_coordinate_descent(
  0%|          | 1/210 [11:42<40:46:36, 702.37s/it]/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 11.61 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/project/cper_neon_aop/conda_envs/py_ml_env/lib/python3.9/site-packages/distributed/client.py:3157: UserWarning: Sending large graph of size 19.87 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
  1%|▏         

In [ ]:
#df_out.to_csv('./results/tmp/vor_bootstrap_by_year_test.csv')